In [1]:
from numba import jit, njit, vectorize

In [2]:
def original_function(input_list):
    output_list = []
    for item in input_list:
        if item % 2 == 0:
            output_list.append(2)
        else:
            output_list.append("1")
    return output_list

In [3]:
test_array = list(range(100_000))

In [4]:
%time _ = original_function(test_array)

CPU times: user 13.7 ms, sys: 4.27 ms, total: 18 ms
Wall time: 17.5 ms


## Numba function

Mixing types may lead to problems, because numba is not able to do.

In [5]:
jitted_function = jit()(original_function)

In [6]:
%time _ = jitted_function(test_array)

<ipython-input-2-9510e5713134>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "original_function" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)) with parameters (Literal[str](1))
 * parameterized
[1] During: resolving callee type: BoundFunction(list.append for list(int64))
[2] During: typing of call at <ipython-input-2-9510e5713134> (7)


File "<ipython-input-2-9510e5713134>", line 7:
def original_function(input_list):
    <source elided>
        else:
            output_list.append("1")
            ^

  def original_function(input_list):
<ipython-input-2-9510e5713134>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "original_function" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-2-9510e5713134>", line 3:
def original_function(input_list):
    <source elided>

CPU times: user 775 ms, sys: 633 ms, total: 1.41 s
Wall time: 721 ms


/home/nicolas/anaconda3/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "original_function" was compiled in object mode without forceobj=True.

File "<ipython-input-2-9510e5713134>", line 3:
def original_function(input_list):
    <source elided>
    output_list = []
    for item in input_list:
    ^

  self.func_ir.loc))
/home/nicolas/anaconda3/lib/python3.7/site-packages/numba/compiler.py:751: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-2-9510e5713134>", line 3:
def original_function(input_list):
    <source elided>
    output_list = []
    for item in input_list:
    ^

  warnings.warn(errors.NumbaDeprecationWarning(msg, self.func_ir.loc))


JIT compilator falls back to __object mode__, generating code trying to figure it out the type, taking longer time than non jit decorated function.

#### __Recommendation__
Do not use the jit decorator by itself but rather the njit decorator or an argumen __jit(nopython=True)__.

In [7]:
jitted_function_no_python = jit(nopython=True)(original_function)

In [8]:
%time _ = jitted_function_no_python(test_array)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of BoundFunction(list.append for list(int64)) with parameters (Literal[str](1))
 * parameterized[0m
[0m[1m[1] During: resolving callee type: BoundFunction(list.append for list(int64))[0m
[0m[1m[2] During: typing of call at <ipython-input-2-9510e5713134> (7)
[0m
[1m
File "<ipython-input-2-9510e5713134>", line 7:[0m
[1mdef original_function(input_list):
    <source elided>
        else:
[1m            output_list.append("1")
[0m            [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
htt

### This time an error instead of a warning is raised.

### Or use njit decorator instead of the argument nopython=True

In [9]:
jitted_function_njit = njit()(original_function)

In [10]:
%time _ = jitted_function_njit(test_array)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of BoundFunction(list.append for list(int64)) with parameters (Literal[str](1))
 * parameterized[0m
[0m[1m[1] During: resolving callee type: BoundFunction(list.append for list(int64))[0m
[0m[1m[2] During: typing of call at <ipython-input-2-9510e5713134> (7)
[0m
[1m
File "<ipython-input-2-9510e5713134>", line 7:[0m
[1mdef original_function(input_list):
    <source elided>
        else:
[1m            output_list.append("1")
[0m            [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
htt

In [11]:
def modified_function(input_list):
    output_list = []
    for item in input_list:
        if item % 2 == 0:
            output_list.append(2)
        else:
            output_list.append(1)
    return output_list

In [12]:
jitted_function_njit_m = njit()(modified_function)

In [13]:
%time _ = modified_function(test_array)

CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 15.2 ms


In [14]:
%time _ = jitted_function_njit_m(test_array)

/home/nicolas/anaconda3/lib/python3.7/site-packages/numba/ir_utils.py:1959: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'input_list' of function 'modified_function'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-11-36e965b83783>", line 1:
def modified_function(input_list):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


CPU times: user 559 ms, sys: 8.21 ms, total: 567 ms
Wall time: 564 ms


It's taking longer...

### Reflection problems
Numba has to figure it out if the type of the array is correct, using reflected lists. Instead __use numpy arrays__ or __numba typed lists__.

In [15]:
import numpy as np

In [16]:
test_np_array = np.arange(10000)

In [17]:
%time _ = modified_function(test_np_array)

CPU times: user 14.2 ms, sys: 0 ns, total: 14.2 ms
Wall time: 13.8 ms


In [18]:
%time _ = jitted_function_njit_m(test_np_array)

CPU times: user 299 ms, sys: 0 ns, total: 299 ms
Wall time: 296 ms


In [19]:
%time _ = jitted_function_njit_m(test_np_array)

CPU times: user 596 µs, sys: 252 µs, total: 848 µs
Wall time: 865 µs


### You see? It's a faster using numpy arrays.

## Numba Vectorized decorator (don't make the compiler lose time figuring out size of arrays)

In [20]:
@vectorize
def scalar_computation(num):
    if num % 2 == 0:
        return 2
    else:
        return 1

In [21]:
%time scalar_computation(test_np_array)

CPU times: user 82.4 ms, sys: 4.69 ms, total: 87.1 ms
Wall time: 85.9 ms


array([2, 1, 2, ..., 1, 2, 1])

In [22]:
@njit
def vectorized_function(input_list):
    output_list = np.zeros_like(input_list)
    for ii, item in enumerate(input_list):
        if item % 2 == 0:
            output_list[ii] = (2)
        else:
            output_list[ii] = (1)
    return output_list

In [23]:
%time _ = vectorized_function(test_np_array)

CPU times: user 182 ms, sys: 0 ns, total: 182 ms
Wall time: 179 ms


## Parallelism in numba

In [37]:
@njit(nogil=True)
def friction_fn(v, vt):
    if v > vt:
        return -v * 3
    else:
        return -vt * 3 * np.sign(v)

@njit(nogil=True)
def simulate_spring_mass_funky_damper(x0, T=10, dt=0.0001, vt=1.0):
    times = np.arange(0, T, dt)
    positions = np.zeros_like(times)
    
    v = 0
    a = 0
    x = x0
    positions[0] = x0/x0
    
    # Instead of enumerate use range (numba works better)
    for ii in range(len(times)):
        if ii == 0:
            continue
        t = times[ii]
        a = friction_fn(v, vt) - 100*x
        v = v + a*dt
        x = x + v*dt
        positions[ii] = x/x0
    return times, positions

In [30]:
%time _ = simulate_spring_mass_funky_damper(1)

CPU times: user 537 ms, sys: 6.92 ms, total: 544 ms
Wall time: 543 ms


In [29]:
# simulate system for different initial conditions -> 8.3h

In [33]:
%time _ = simulate_spring_mass_funky_damper(1)

CPU times: user 1.7 ms, sys: 0 ns, total: 1.7 ms
Wall time: 1.74 ms


In [36]:
%%time
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(8) as ex:
    ex.map(simulate_spring_mass_funky_damper, np.arange(0.1, 1000))

CPU times: user 2.23 s, sys: 40.8 ms, total: 2.27 s
Wall time: 2.22 s


### Numba by default does not release the GIL, so using threads does not bring any benefit.

### You can use nogil=True as an argument to tell numba to release the GIL.

In [40]:
%%time
from concurrent.futures import ThreadPoolExecutor

n_cores = 8

with ThreadPoolExecutor(n_cores) as ex:
    ex.map(simulate_spring_mass_funky_damper, np.arange(0.1, 1000))

CPU times: user 2.14 s, sys: 57 ms, total: 2.2 s
Wall time: 435 ms


### Don't actually have to use ThreadPoolExecutor directly but use an argument in the decorator named __parallel=True__

In [46]:
from numba import prange

@njit(parallel=True)
def run_sims(end=1000):
    for ii in prange(int(end/0.1)):
        if ii == 0:
            continue
        simulate_spring_mass_funky_damper(ii*0.1)

In [49]:
%time run_sims(1000)

CPU times: user 23.2 s, sys: 112 ms, total: 23.3 s
Wall time: 3.27 s


## Note:

Use __htop__ to manage and visualize processes running on the CPU cores.